In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import timeit
from sklearn.cluster import KMeans
from sklearn.neighbors import KDTree
from sklearn import datasets

In [26]:
clusters = {}
centroids = []
np_data = []
total_for_coreset = 0
#function to generate 'num_points' random points of 'dim' dimensions.
def generate_data(data, m):
	#if data_type == 1:
	#	pass
	#filename = sys.argv[1] #dataset to calculate coreset of
	#output = sys.argv[2] #output file to print probability distribution values
	dim = data.shape[1]
	rows = data.shape[0]
	data_mean = np.mean(data, axis = 0)
	#print(data_mean)
	distance = 0
	for point in data:
		distance += np.sum(np.square(point-data_mean))
	#print(distance)
	prob_dist = []
	#calculating proposal  distribution for each row
	for point in data:
		value=((0.5*(1/n))+0.5*((np.sum(np.square(point-data_mean)))/distance))
		prob_dist.append(value)
	df = pd.DataFrame(prob_dist)
	data = pd.DataFrame(data)
	data['Prob_dist'] = df
	weight_value = []
	for i in range(n):
		weight = 1/(m*prob_dist[i])
		weight_value.append(weight)
	df = pd.DataFrame(weight_value)
	data['weight_value'] = df
	sampled_points = data.sample(n=m, frac=None, replace=False, weights='weight_value', random_state=None, axis=None)
	return sampled_points.iloc[:,:dim]

In [27]:
# Loading data
data = datasets.load_wine()

# Identifying the number of classes
k = data.target_names.size
data = pd.DataFrame(data.data)
data = np.array(data)

# Identifying data length
n = data.shape[0]

## Finding the number of attributes.
features = data.shape[1]


start = timeit.default_timer()
points = np.array(generate_data(data, k))
stop = timeit.default_timer()
total_for_coreset = stop-start

In [28]:
## Running KMeans on train data.
start = timeit.default_timer()
y_predict = KMeans(n_clusters = k, init = points, n_init =1).fit(data)
stop = timeit.default_timer()
time_for_kmeans = stop-start

In [29]:
# No. of. Nearest Neighbors
n_n = 7

## Storing Cluster centers in cluster_centers
cluster_centers = np.array(y_predict.cluster_centers_)

## generating Query point
query = []
for i in range(0,features):
    x = round(random.random()*random.choice([0,1,100]),1)
    query.append(x)


#query = []
#for i in range(0,features):
#    x = round(random.random()*random.choice([1,0,100,1000]),1)
#    query.append(x)

query = np.array(query)
query = query.reshape(1,features)


## Finding the nearest cluster to query point.
    ## Finding the euclidean distance from first cluster_center point.
start = timeit.default_timer()
dist = 0
for i in range(features):
   dist += (query[0,i]-cluster_centers[0,i])**2
dist = np.sqrt(dist)
near_cluster = 0
i = 1

 ## Finding the euclidean distance from other cluster_center points.
while i < k:
    dist_j = 0
    for j in range(features):
       dist_j += (query[0,j]-cluster_centers[i,j])**2
    dist_j = np.sqrt(dist_j)
    if(dist_j < dist):
        dist = dist_j
        near_cluster = i
    i += 1
stop = timeit.default_timer()   
time_for_closest_center = stop-start   
## Finding the cluster elements to construct KD-Tree

cluster_values = np.ones([n,features])
j = 0
for i in range(n):
    if y_predict.labels_[i] == near_cluster:
        cluster_values[j] = data[i]
        j += 1
cluster_values = cluster_values[:j,:j]

## KDTree Construction and Querying
start = timeit.default_timer()
tree = KDTree(cluster_values)
stop = timeit.default_timer()
time_for_kdtree = stop-start

start = timeit.default_timer()
dist_main , ind_main = tree.query(query[0:1],n_n)
stop = timeit.default_timer()
query_time = stop-start


j=0
for i in ind_main:
    print(cluster_values[i], dist_main[j])
    j += 1
print(total_for_coreset,time_for_kmeans,time_for_closest_center,time_for_kdtree,query_time)
print(total_for_coreset + time_for_kmeans + time_for_closest_center + time_for_kdtree + query_time)

[[1.200e+01 9.200e-01 2.000e+00 1.900e+01 8.600e+01 2.420e+00 2.260e+00
  3.000e-01 1.430e+00 2.500e+00 1.380e+00 3.120e+00 2.780e+02]
 [1.229e+01 2.830e+00 2.220e+00 1.800e+01 8.800e+01 2.450e+00 2.250e+00
  2.500e-01 1.990e+00 2.150e+00 1.150e+00 3.300e+00 2.900e+02]
 [1.222e+01 1.290e+00 1.940e+00 1.900e+01 9.200e+01 2.360e+00 2.040e+00
  3.900e-01 2.080e+00 2.700e+00 8.600e-01 3.020e+00 3.120e+02]
 [1.242e+01 2.550e+00 2.270e+00 2.200e+01 9.000e+01 1.680e+00 1.840e+00
  6.600e-01 1.420e+00 2.700e+00 8.600e-01 3.300e+00 3.150e+02]
 [1.252e+01 2.430e+00 2.170e+00 2.100e+01 8.800e+01 2.550e+00 2.270e+00
  2.600e-01 1.220e+00 2.000e+00 9.000e-01 2.780e+00 3.250e+02]
 [1.237e+01 1.630e+00 2.300e+00 2.450e+01 8.800e+01 2.220e+00 2.450e+00
  4.000e-01 1.900e+00 2.120e+00 8.900e-01 2.780e+00 3.420e+02]
 [1.162e+01 1.990e+00 2.280e+00 1.800e+01 9.800e+01 3.020e+00 2.260e+00
  1.700e-01 1.350e+00 3.250e+00 1.160e+00 2.960e+00 3.450e+02]] [295.98884455 307.78430759 329.76853974 332.21101336 3

In [30]:
start = timeit.default_timer()
tree = KDTree(data)
stop = timeit.default_timer()
total_time = stop-start
print(total_time)
j=0
start = timeit.default_timer()
dist_dum , ind_dum = tree.query(query,n_n)
stop = timeit.default_timer()
total_time = stop-start
print(total_time)
for i in ind_dum:
    print(data[i],dist_dum[j])
    j =+ 1

0.00038041900006646756
0.0003723290010384517
[[1.200e+01 9.200e-01 2.000e+00 1.900e+01 8.600e+01 2.420e+00 2.260e+00
  3.000e-01 1.430e+00 2.500e+00 1.380e+00 3.120e+00 2.780e+02]
 [1.229e+01 2.830e+00 2.220e+00 1.800e+01 8.800e+01 2.450e+00 2.250e+00
  2.500e-01 1.990e+00 2.150e+00 1.150e+00 3.300e+00 2.900e+02]
 [1.222e+01 1.290e+00 1.940e+00 1.900e+01 9.200e+01 2.360e+00 2.040e+00
  3.900e-01 2.080e+00 2.700e+00 8.600e-01 3.020e+00 3.120e+02]
 [1.242e+01 2.550e+00 2.270e+00 2.200e+01 9.000e+01 1.680e+00 1.840e+00
  6.600e-01 1.420e+00 2.700e+00 8.600e-01 3.300e+00 3.150e+02]
 [1.252e+01 2.430e+00 2.170e+00 2.100e+01 8.800e+01 2.550e+00 2.270e+00
  2.600e-01 1.220e+00 2.000e+00 9.000e-01 2.780e+00 3.250e+02]
 [1.237e+01 1.630e+00 2.300e+00 2.450e+01 8.800e+01 2.220e+00 2.450e+00
  4.000e-01 1.900e+00 2.120e+00 8.900e-01 2.780e+00 3.420e+02]
 [1.162e+01 1.990e+00 2.280e+00 1.800e+01 9.800e+01 3.020e+00 2.260e+00
  1.700e-01 1.350e+00 3.250e+00 1.160e+00 2.960e+00 3.450e+02]] [295.9888

In [31]:
query

array([[ 0. ,  0. ,  0.6,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,
        52.7,  0.1]])

In [32]:
## Running KMeans on train data.
start = timeit.default_timer()
y_predict = KMeans(n_clusters = k).fit(data)
stop = timeit.default_timer()
time_for_kmeans = stop-start


## Storing Cluster centers in cluster_centers
cluster_centers = np.array(y_predict.cluster_centers_)

## Finding the nearest cluster to query point.
    ## Finding the euclidean distance from first cluster_center point.
start = timeit.default_timer()
dist = 0
for i in range(features):
   dist += (query[0,i]-cluster_centers[0,i])**2
dist = np.sqrt(dist)
near_cluster = 0
i = 1

 ## Finding the euclidean distance from other cluster_center points.
while i < k:
    dist_j = 0
    for j in range(features):
       dist_j += (query[0,j]-cluster_centers[i,j])**2
    dist_j = np.sqrt(dist_j)
    if(dist_j < dist):
        dist = dist_j
        near_cluster = i
    i += 1
    
stop = timeit.default_timer()   
time_for_closest_center = stop-start
## Finding the cluster elements to construct KD-Tree
cluster_values = np.ones([n,features])
j = 0
for i in range(n):
    if y_predict.labels_[i] == near_cluster:
        cluster_values[j] = data[i]
        j += 1
cluster_values = cluster_values[:j,:j]

## KDTree Construction and Querying
start = timeit.default_timer()
tree = KDTree(cluster_values)
stop = timeit.default_timer()
time_for_kdtree = stop-start

start = timeit.default_timer()
dist_main , ind_main = tree.query(query[0:1],n_n)
stop = timeit.default_timer()
query_time = stop-start


j=0
for i in ind_main:
    print(cluster_values[i], dist_main[j])
    j += 1
print(time_for_kmeans,time_for_closest_center,time_for_kdtree,query_time)
print(time_for_closest_center + time_for_kdtree + query_time)

[[1.200e+01 9.200e-01 2.000e+00 1.900e+01 8.600e+01 2.420e+00 2.260e+00
  3.000e-01 1.430e+00 2.500e+00 1.380e+00 3.120e+00 2.780e+02]
 [1.229e+01 2.830e+00 2.220e+00 1.800e+01 8.800e+01 2.450e+00 2.250e+00
  2.500e-01 1.990e+00 2.150e+00 1.150e+00 3.300e+00 2.900e+02]
 [1.222e+01 1.290e+00 1.940e+00 1.900e+01 9.200e+01 2.360e+00 2.040e+00
  3.900e-01 2.080e+00 2.700e+00 8.600e-01 3.020e+00 3.120e+02]
 [1.242e+01 2.550e+00 2.270e+00 2.200e+01 9.000e+01 1.680e+00 1.840e+00
  6.600e-01 1.420e+00 2.700e+00 8.600e-01 3.300e+00 3.150e+02]
 [1.252e+01 2.430e+00 2.170e+00 2.100e+01 8.800e+01 2.550e+00 2.270e+00
  2.600e-01 1.220e+00 2.000e+00 9.000e-01 2.780e+00 3.250e+02]
 [1.237e+01 1.630e+00 2.300e+00 2.450e+01 8.800e+01 2.220e+00 2.450e+00
  4.000e-01 1.900e+00 2.120e+00 8.900e-01 2.780e+00 3.420e+02]
 [1.162e+01 1.990e+00 2.280e+00 1.800e+01 9.800e+01 3.020e+00 2.260e+00
  1.700e-01 1.350e+00 3.250e+00 1.160e+00 2.960e+00 3.450e+02]] [295.98884455 307.78430759 329.76853974 332.21101336 3